In [ ]:
# Ok, so for the media library, we need to prepare the output in a certain way. 
# input will be a bunch of ids
# Add these to an album


# when render_album is called, it will take the album id, process all the media in that library, and then return object in correct format


# Vector_Search currently returns data in this format
# {'id': 127, 'content': 'a lion cub sitting on a rock under a tree', 'url': None, 'similarity': 0.933864435633929}
# {'id': 121, 'content': 'a young lion walking in the grass next to a tree', 'url': None, 'similarity': 0.908780846784397}
# {'id': 111, 'content': 'a close up of a lion in a field', 'url': None, 'similarity': 0.908208218035866}
# {'id': 119, 'content': 'a lion standing in the middle of a forest', 'url': None, 'similarity': 0.900758166540314}
# {'id': 114, 'content': 'a close up of a lion laying on the ground', 'url': None, 'similarity': 0.899268792635519}
# {'id': 118, 'content': 'a large lion laying on top of a rock', 'url': None, 'similarity': 0.897526972442822}
# {'id': 117, 'content': 'a lion behind a fence in a zoo', 'url': None, 'similarity': 0.897353962723208}
# {'id': 112, 'content': 'a lion walking across a lush green field', 'url': None, 'similarity': 0.895988907185434}
# {'id': 113, 'content': 'a lion standing on top of a large rock', 'url': None, 'similarity': 0.893444278649301}


# table album_media_link
# id | album_id | media_id

In [ ]:
import funcs_supabase

media_id_list = [80,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104]
album_id = 12
table = 'album_media_link'

resp_list = []

for media_id in media_id_list:
    combined_dict = {}

    combined_dict['album_id'] = int(album_id)
    combined_dict['media_id'] = int(media_id)
    print(combined_dict)


    conditions = [
        {"column": "media_id", "operator": "eq", "value": media_id},
        {"column": "album_id", "operator": "eq", "value": album_id}
    ]
    check = funcs_supabase.select_data_with_conditions(table, conditions)
    print(check)

    if len(check) == 0:
        print(f"Media_id {media_id} does not exist in album_id {album_id}")

        try:
            resp = funcs_supabase.insert_data(table, combined_dict)
            resp_list.append(resp)

        except Exception as e:
            print(f"Failed to insert data for media_id {media_id}. Error: {e}")
    #         resp_list.append(f"Failed: {e}")
    
    # else:
    #     print(f"Media_id {media_id} already exists in album_id {album_id}")

print(resp_list)


In [ ]:
import psycopg2
import os
import psycopg2.extras  # Import the extras
from dotenv import load_dotenv

def get_media_by_album_id(album_id):
    # Load environment variables
    load_dotenv()

    # Get database connection string from environment variables
    SUPABASE_DB_FULL_URL = os.getenv("SUPABASE_DB_FULL_URL")

    # Initialize connection and cursor
    conn = psycopg2.connect(SUPABASE_DB_FULL_URL)
    cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

    # SQL query
    sql_query = """
    SELECT media.*
    FROM album_media_link
    JOIN media ON album_media_link.media_id = media.id
    WHERE album_media_link.album_id = %s;
    """

    result_list = []

    try:
        # Execute the query
        cursor.execute(sql_query, (album_id,))

        # Fetch all rows and store them in a list of dictionaries
        rows = cursor.fetchall()
        for row in rows:
            result_list.append(dict(row))

    except Exception as e:
        print(f"An error occurred while executing the SQL query: {e}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        conn.close()

    return result_list

# Test the function
album_id_to_search = 10  # Replace with your actual album_id
result = get_media_by_album_id(album_id_to_search)
print(result)  # This will print a list of dictionaries


In [ ]:

def get_cdn_link(url, width = None, height = None, resize = None):
    # resize: 'contain', // 'cover' | 'fill'
    # https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/4b8840f6-1d1a-4b78-b728-fcf039a2f366.jpeg    
    # https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/render/image/public/media/0bd35f13-e222-4504-9362-5183bda46a6b.png?width=300&height=300&resize=cover
    # parsed_url = urlparse(url)
    # print(f"Scheme: {parsed_url.scheme}")
    # print(f"Netloc: {parsed_url.netloc}")
    # print(f"Path: {parsed_url.path}")
    # print(f"Parameters: {parsed_url.params}")
    # print(f"Query: {parsed_url.query}")
    # print(f"Fragment: {parsed_url.fragment}")

    replace_me = "https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/"
    replace_with = "https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/render/image/public/media/"

    new_url = url.replace(replace_me, replace_with)
    # remove all the parameters
    new_url = new_url.split("?")[0]

    new_params = []

    if width is not None:
        new_params.append(f"width={width}")

    if height is not None:
        new_params.append(f"height={height}")

    if resize is not None:
        new_params.append(f"resize={resize}")

    if len(new_params) > 0:
        new_url = new_url + "?" + "&".join(new_params)

    # print(new_url)

    return new_url




def process_media_list_for_display(media_list):
    # print(media_list)

    output_list = []

    for media in media_list:
        # id, media_type, description, url, caption, thumbnail, source_url, source_type
        media_dict = {}
        
        media_dict['id'] = media['id']

        media_dict['media_type'] = media['type']

        if media['description'] is not None:
            media_dict['description'] = media['description']

        if media["url"] is None:
            if media["url_remote"] is not None:
                media_dict['url'] = media["url_remote"]

        else:
            media_dict['url'] = media["url"]

        if media["caption"] is not None:
            media_dict['caption'] = media["caption"]

        if media["thumbnail_url"] is not None:
            try:
                cdn_link = get_cdn_link(media["thumbnail_url"], width = 450, resize = "contain")
                media_dict['thumbnail'] = cdn_link
                
            except:
                media_dict['thumbnail'] = media["thumbnail_url"]


        if media["source_type"] is not None:
            media_dict['source_type'] = media["source_type"]


        if media["source_data"]["url"] is not None:
            media_dict['source_url'] = media["source_data"]["url"]


        output_list.append(media_dict)


    return output_list
        # print(media)


# Test the function
album_id_to_search = 10  # Replace with your actual album_id
result = get_media_by_album_id(album_id_to_search)
print(result)  # This will print a list of dictionaries

processed_result = process_media_list_for_display(result)
print(processed_result)  # This will print a list of dictionaries



In [ ]:
# Search all keywords in a scene or project. 
# Save results to db. 
# Semantic search the results.
# Add best ones to an album. 


In [27]:
import funcs_supabase
import pexels


def import_pexels_for_project(project_id, scene_id=None):
    # Fetch scenes from the database based on the project_id
    scenes = funcs_supabase.select_data('scenes', 'project_id', project_id)

    # Loop through each scene to process
    for scene in scenes:
        # Extract various properties from each scene
        scene_keywords = scene['theme_keywords']
        album_id = scene['album_id']
        scene_text = scene['scene_text']

        # Make sure that scene_keywords is a list
        if type(scene_keywords) != list:
            continue  # Skip this iteration if scene_keywords is not a list

        # If a specific scene_id is given, only process that scene
        if scene_id is not None:
            if scene_id == scene['id']:
                print("scene_id matches")
                pexels.process_pexels_kw_list(scene_keywords)
                break  # Exit the loop as we've found and processed the matching scene
            else:
                continue  # Skip this iteration as the scene_id doesn't match

        # If no specific scene_id is provided, process all scenes
        else:
            pexels.process_pexels_kw_list(scene_keywords)


project_id = 18
# my_scene_id = 100
import_pexels_for_project(project_id)





2023-09-12 20:12:23,774:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.18 "HTTP/1.1 200 OK"


scene_id matches


2023-09-12 20:12:24,468:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media?select=%2A&source_id=eq.2022396 "HTTP/1.1 200 OK"
2023-09-12 20:12:24,567:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media?select=%2A&source_id=eq.3941289 "HTTP/1.1 200 OK"


The list is not empty. Stop
The list is not empty. Stop


2023-09-12 20:12:24,676:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media?select=%2A&source_id=eq.1692701 "HTTP/1.1 200 OK"
2023-09-12 20:12:24,776:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media?select=%2A&source_id=eq.3722009 "HTTP/1.1 200 OK"
2023-09-12 20:12:24,846:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media?select=%2A&source_id=eq.4729195 "HTTP/1.1 200 OK"


...

uploaded


2023-09-12 20:24:22,089:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media "HTTP/1.1 201 Created"


a hand holding a remote control in front of a television
{'dimensions': {'width': 1200, 'height': 630, 'channels': 3}, 'mean_color': (61.243267195767196, 63.87573015873016, 72.737917989418, 0.0), 'file_size': 31336, 'file_type': 'jpeg'}
Media data updated


2023-09-12 20:24:22,588:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/vectors "HTTP/1.1 201 Created"
2023-09-12 20:24:22,680:INFO - HTTP Request: PATCH https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media?id=eq.406 "HTTP/1.1 200 OK"


Vector data updated
image processed
Successfully processed 126 items.


In [32]:
import vector_search
# returns the most similar vectors from the vectors table


def vector_search_results(scene_text_embedding):
    vector_search_results =  vector_search.search_vectors(scene_text_embedding, match_threshold=0.8, match_count=20)

    media_table_id_list = []

    for res in vector_search_results:
        print(res)
        media_table_id_list.append(res['media_table_id'])

    return media_table_id_list




In [31]:
def link_media_to_album(album_id: int, media_id_list: list) -> list:
    "attaches  a list of media ids to an album"

    table = 'album_media_link'
    resp_list = []
    
    for media_id in media_id_list:
        combined_dict = {}
        combined_dict['album_id'] = int(album_id)
        combined_dict['media_id'] = int(media_id)
        print(combined_dict)

        conditions = [
            {"column": "media_id", "operator": "eq", "value": media_id},
            {"column": "album_id", "operator": "eq", "value": album_id}
        ]
        check = funcs_supabase.select_data_with_conditions(table, conditions)
        print(check)

        if len(check) == 0:
            print(f"Media_id {media_id} does not exist in album_id {album_id}")

            try:
                resp = funcs_supabase.insert_data(table, combined_dict)
                resp_list.append(resp)
            except Exception as e:
                print(f"Failed to insert data for media_id {media_id}. Error: {e}")
        # else:
        #     print(f"Media_id {media_id} already exists in album_id {album_id}")

    return resp_list

# # Example usage
# album_id = 12
# media_id_list = [80, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]
# result = link_media_to_album(album_id, media_id_list)
# print(result)


# result = link_media_to_album(album_id, media_table_id_list)



In [33]:
scenes = funcs_supabase.select_data('scenes', 'project_id', project_id)
for scene in scenes:
    print(scene)

    scene_text = scene['scene_text']
    album_id = scene['album_id']
    print(scene_text)
    scene_text_embedding = vector_search.get_embeddings(scene_text)

    media_id_list = vector_search_results(scene_text_embedding)
    print(media_id_list)
    link_media_to_album(album_id, media_id_list)
    

2023-09-12 22:10:16,910:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.18 "HTTP/1.1 200 OK"


{'id': 97, 'created_at': '2023-09-12T16:39:36.771468+00:00', 'project_id': 18, 'scene_order_position': 1, 'uuid': '83ca5db9-1f43-4c80-92cd-37b53912b61c', 'scene_text': 'She decided to restore it, feeling an inexplicable connection to the instrument.', 'status': 'Active', 'album_id': 111, 'theme_keywords': ['Instrument restoration', 'Feeling of connection', 'Preserving the past', 'Craftsmanship', 'Emotional attachment']}
She decided to restore it, feeling an inexplicable connection to the instrument.


2023-09-12 22:10:17,692:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link?select=%2A&media_id=eq.351&album_id=eq.111 "HTTP/1.1 200 OK"


{'id': 370, 'content': 'a man holding a violin in her right hand', 'media_table_id': 351, 'similarity': 0.815303287632657}
{'id': 268, 'content': 'a man sitting in front of a table with a violin', 'media_table_id': 249, 'similarity': 0.814724169824482}
{'id': 222, 'content': 'a violin leaning against a log in a field', 'media_table_id': 203, 'similarity': 0.810716992523859}
{'id': 363, 'content': 'a man holding a violin while standing next to a microphone', 'media_table_id': 344, 'similarity': 0.80930611421913}
{'id': 265, 'content': 'a man holding a violin in his right hand', 'media_table_id': 246, 'similarity': 0.802805233606894}
{'id': 350, 'content': 'a close up of a person playing a violin', 'media_table_id': 331, 'similarity': 0.800232775523766}
{'id': 352, 'content': 'a man playing a violin in a dark room', 'media_table_id': 333, 'similarity': 0.800097795415784}
[351, 249, 203, 344, 246, 331, 333]
{'album_id': 111, 'media_id': 351}
[]
Media_id 351 does not exist in album_id 111


2023-09-12 22:10:17,794:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-09-12 22:10:17,904:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link?select=%2A&media_id=eq.249&album_id=eq.111 "HTTP/1.1 200 OK"


...

[]
Media_id 191 does not exist in album_id 110
{'album_id': 110, 'media_id': 195}
[]
Media_id 195 does not exist in album_id 110


2023-09-12 22:10:35,422:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link?select=%2A&media_id=eq.185&album_id=eq.110 "HTTP/1.1 200 OK"
2023-09-12 22:10:35,510:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"


{'album_id': 110, 'media_id': 185}
[]
Media_id 185 does not exist in album_id 110
{'album_id': 110, 'media_id': 193}


2023-09-12 22:10:35,606:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link?select=%2A&media_id=eq.193&album_id=eq.110 "HTTP/1.1 200 OK"
2023-09-12 22:10:35,671:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"


[]
Media_id 193 does not exist in album_id 110


In [ ]:
import funcs_supabase

def delete_album_media_link_ids(ids_list):
    """
    Deletes album media links based on their IDs.

    :param ids_list: List of IDs to delete.
    :return: List of responses from delete action.
    """
    return [funcs_supabase.delete_data("album_media_link", id_) for id_ in ids_list]


def find_album_media_ids(album_id, media_id):
    """
    Find IDs in album_media_link table for a given album and media ID.

    :param album_id: ID of the album.
    :param media_id: ID of the media.
    :return: List of IDs that match the conditions.
    """
    conditions = [
        {"column": "album_id", "operator": "eq", "value": album_id},
        {"column": "media_id", "operator": "eq", "value": media_id}
    ]
    records = funcs_supabase.select_data_with_conditions("album_media_link", conditions)
    return [item['id'] for item in records]


def remove_media_from_album(album_id, media_id=None):
    """
    Removes media from an album. If media_id is None, it removes all media from the album.

    :param album_id: ID of the album.
    :param media_id: ID(s) of the media to remove.
    """
    if media_id is None:
        # Remove all media from the album
        records = funcs_supabase.select_data("album_media_link", "album_id", album_id)
        ids_list = [item['id'] for item in records]
        delete_album_media_link_ids(ids_list)

    elif isinstance(media_id, int):
        # Remove a single media from the album
        ids_list = find_album_media_ids(album_id, media_id)
        delete_album_media_link_ids(ids_list)

    elif isinstance(media_id, list):
        # Remove multiple media items from the album
        ids_list = [id_ for media in media_id for id_ in find_album_media_ids(album_id, media)]
        delete_album_media_link_ids(ids_list)

    else:
        print(f"Unsupported type for media_id: {type(media_id)}")


# # Sample usage
# album_id = 110
# result = remove_media_from_album(album_id)
# print(result)
